# Proceso de ETL  del Dataset 'australian_user_reviews.json'

In [347]:
# Importar las librerias necesarias
import pandas as pd
import ast

%load_ext autoreload
%autoreload 2
import utils

import warnings
warnings.filterwarnings("ignore")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Extraccion de datos del Dataset 'australian_user_reviews'

In [348]:
# Leemos el archivo 'australian_users_reviews.json' y creamos el DataFrame directamente
with open('../Datasets/australian_user_reviews.json', 'r', encoding='utf-8') as f:
    df_user_reviews = pd.DataFrame([ast.literal_eval(line) for line in f])
# Desplegamos el DataFrame
df_user_reviews

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


In [349]:
# Funcion que devuelve los tipos de datos, ademas de la presencia y/o ousencia de valores nulos en un DataFrame
def verificar_tipo_datos(df):
    # Crear un diccionario para almacenar la informacion acerca del DataFrame analizado
    dic = {
        "nombre_campo": df.columns,
        "tipo_datos": [df[col].apply(type).unique() for col in df.columns],
        "no_nulos_%": [round(df[col].count() / len(df) * 100, 2) for col in df.columns],
        "nulos_%": [round((1 - df[col].count() / len(df)) * 100, 2) for col in df.columns],
        "nulos": [df[col].isnull().sum() for col in df.columns]
    }
    # Convertir dic en un DataFrame
    info = pd.DataFrame(dic)
    # Retornar valores del Dataframe info
    return info

In [350]:
verificar_tipo_datos(df_user_reviews)

,nombre_campo,tipo_datos,no_nulos_%,nulos_%,nulos
0,user_id,[<class 'str'>],100.0,0.0,0
1,user_url,[<class 'str'>],100.0,0.0,0
2,reviews,[<class 'list'>],100.0,0.0,0


In [351]:
# Funcion que verifica y muestra filas duplicadas en un DataFrame basado en una columna específica
# Recibe como argumentos un Dataframe y la Columna que va ser analizada
def verifica_duplicados_por_columna(df, columna):
    # Filtrar y ordenar las filas duplicadas
    df_duplicados = df[df.duplicated(subset=columna, keep=False)].sort_values(by=columna)

    # Devolver el DataFrame de duplicados o None si no hay duplicados
    return df_duplicados if not df_duplicados.empty else None

In [352]:
# Verificar si hay duplicados en la columna
filas_duplicadas = verifica_duplicados_por_columna(df_user_reviews, 'user_id')
# Desplegar el DataFrame con las filas duplicadas
filas_duplicadas

,user_id,user_url,reviews
12888,05041129,http://steamcommunity.com/id/05041129,"[{'funny': '', 'posted': 'Posted May 18, 2015...."
5250,05041129,http://steamcommunity.com/id/05041129,"[{'funny': '', 'posted': 'Posted May 18, 2015...."
3133,111222333444555666888,http://steamcommunity.com/id/11122233344455566...,"[{'funny': '', 'posted': 'Posted December 22, ..."
3134,111222333444555666888,http://steamcommunity.com/id/11122233344455566...,"[{'funny': '', 'posted': 'Posted December 22, ..."
4139,29123,http://steamcommunity.com/id/29123,"[{'funny': '', 'posted': 'Posted March 26.', '..."
...,...,...,...
2721,xXAussieRockXx,http://steamcommunity.com/id/xXAussieRockXx,"[{'funny': '', 'posted': 'Posted July 17, 2015..."
2680,yolofaceguy,http://steamcommunity.com/id/yolofaceguy,"[{'funny': '', 'posted': 'Posted October 31, 2..."
17916,yolofaceguy,http://steamcommunity.com/id/yolofaceguy,"[{'funny': '', 'posted': 'Posted October 31, 2..."
5855,zeroblade,http://steamcommunity.com/id/zeroblade,"[{'funny': '', 'posted': 'Posted November 30, ..."


In [353]:
# Usuario de ejemplo
user_id = '29123'
# Filtrar las filas duplicadas para obtener solo las reseñas del usuario específico
user_reviews = filas_duplicadas[filas_duplicadas['user_id'] == user_id]['reviews']
# Iterar sobre las listas de reseñas del usuario
for review_list in user_reviews:
    # Iterar sobre cada reseña en la lista
    for review in review_list:
        # Desplegar el contenido de cada reseña
        print(review['review'])
    print('-' * 40)

Can't play after the updates, people who doesn't know how to play the game, after playing the game crashes and The ♥♥♥♥ing black screen
5/10 lots of bugs and bad servers.BTW this game needs a better training facility
it needs blood when get hit, and some others changes, 9/10
What can i say? Nice graphics and nice story and charactersThe only thing wrong: it should be playable for all graphics cards even the lowest ones like 12 fps or 6But nvm nice game 8/10
Well, this game is the best i ever played but when i was downloading this game (again) i was like in 30%, but i connected today and it was 0% well if valve can fix this things i will be more happy and i will play more this game.
----------------------------------------
Can't play after the updates, people who doesn't know how to play the game, after playing the game crashes and The ♥♥♥♥ing black screen
5/10 lots of bugs and bad servers.BTW this game needs a better training facility
it needs blood when get hit, and some others change

In [354]:
# Verificar datos duplicados en la columna 'user_id'
df_user_items = df_user_items.drop_duplicates(subset='user_id', keep='first')
filas_duplicadas = verifica_duplicados_por_columna(df_user_items, 'user_id')
# Desplegar los duplicados
print(filas_duplicadas)

None


In [355]:
# Ver el tipo de dato que contiene la columna 'review'
df_user_items['reviews'][0]

[{'funny': '',
  'posted': 'Posted November 5, 2011.',
  'last_edited': '',
  'item_id': '1250',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': 'Simple yet with great replayability. In my opinion does "zombie" hordes and team work better than left 4 dead plus has a global leveling system. Alot of down to earth "zombie" splattering fun for the whole family. Amazed this sort of FPS is so rare.'},
 {'funny': '',
  'posted': 'Posted July 15, 2011.',
  'last_edited': '',
  'item_id': '22200',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': "It's unique and worth a playthrough."},
 {'funny': '',
  'posted': 'Posted April 21, 2011.',
  'last_edited': '',
  'item_id': '43110',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': 'Great atmosphere. The gunplay can be a bit chunky at times but at the end of the day this game is definitely worth it and I hope they do a sequel...so buy the game so I get a sequel!'}]

# Este conjunto contiene 3 columnas y 25799 filas, sin valores nulos. Las columnas son:

user_id: identificador único para el usuario.
user_url: url del perfil del usuario en streamcommunity.
reviews: lista de diccionarios, cada usuario tiene uno o mas diccionario con el review, cada diccionario contiene:
 - funny: indica si alguien puso emoticón de gracioso al review.
 - posted: fecha de posteo del review en formato Posted April 21, 2011.
 - last_edited: fecha de la última edición.
 - item_id: identificador único del item, es decir, del juego.
 - helpful: estadística donde otros usuarios indican si fue útil la información.
 - recommend: booleano que indica si el usuario recomienda o no el juego.
 - review: sentencia string con los comentarios sobre el juego.

# Transformar la columna 'reviews'

In [356]:
# Transformar a columnas cada elemento de las listas
df_user_items2 = pd.json_normalize(df_user_items['reviews'])
df_user_items2.head()

,0,1,2,3,4,5,6,7,8,9
0,"{'funny': '', 'posted': 'Posted November 5, 20...","{'funny': '', 'posted': 'Posted July 15, 2011....","{'funny': '', 'posted': 'Posted April 21, 2011...",None,None,None,None,None,None,None
1,"{'funny': '', 'posted': 'Posted June 24, 2014....","{'funny': '', 'posted': 'Posted September 8, 2...","{'funny': '', 'posted': 'Posted November 29, 2...",None,None,None,None,None,None,None
2,"{'funny': '', 'posted': 'Posted February 3.', ...","{'funny': '', 'posted': 'Posted December 4, 20...","{'funny': '', 'posted': 'Posted November 3, 20...","{'funny': '', 'posted': 'Posted October 15, 20...","{'funny': '', 'posted': 'Posted October 15, 20...","{'funny': '', 'posted': 'Posted October 15, 20...",None,None,None,None
3,"{'funny': '', 'posted': 'Posted October 14, 20...","{'funny': '', 'posted': 'Posted July 28, 2012....","{'funny': '', 'posted': 'Posted June 2, 2012.'...","{'funny': '', 'posted': 'Posted June 29, 2014....","{'funny': '', 'posted': 'Posted November 22, 2...","{'funny': '', 'posted': 'Posted February 23, 2...",None,None,None,None
4,"{'funny': '3 people found this review funny', ...","{'funny': '1 person found this review funny', ...","{'funny': '2 people found this review funny', ...","{'funny': '', 'posted': 'Posted July 11, 2013....",None,None,None,None,None,None


In [357]:
# Agregar el 'user_id' y 'user_url' a las columnas separadas 
df_user_items2 = pd.concat([df_user_items[['user_id', 'user_url']], df_user_items2], axis=1)
df_user_items2.head()

,user_id,user_url,0,1,2,3,4,5,6,7,8,9
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted November 5, 20...","{'funny': '', 'posted': 'Posted July 15, 2011....","{'funny': '', 'posted': 'Posted April 21, 2011...",None,None,None,None,None,None,None
1,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted June 24, 2014....","{'funny': '', 'posted': 'Posted September 8, 2...","{'funny': '', 'posted': 'Posted November 29, 2...",None,None,None,None,None,None,None
2,evcentric,http://steamcommunity.com/id/evcentric,"{'funny': '', 'posted': 'Posted February 3.', ...","{'funny': '', 'posted': 'Posted December 4, 20...","{'funny': '', 'posted': 'Posted November 3, 20...","{'funny': '', 'posted': 'Posted October 15, 20...","{'funny': '', 'posted': 'Posted October 15, 20...","{'funny': '', 'posted': 'Posted October 15, 20...",None,None,None,None
3,doctr,http://steamcommunity.com/id/doctr,"{'funny': '', 'posted': 'Posted October 14, 20...","{'funny': '', 'posted': 'Posted July 28, 2012....","{'funny': '', 'posted': 'Posted June 2, 2012.'...","{'funny': '', 'posted': 'Posted June 29, 2014....","{'funny': '', 'posted': 'Posted November 22, 2...","{'funny': '', 'posted': 'Posted February 23, 2...",None,None,None,None
4,maplemage,http://steamcommunity.com/id/maplemage,"{'funny': '3 people found this review funny', ...","{'funny': '1 person found this review funny', ...","{'funny': '2 people found this review funny', ...","{'funny': '', 'posted': 'Posted July 11, 2013....",None,None,None,None,None,None


In [358]:
# Selecciona las columnas a derretir directamente, sin necesidad de range(9)
columns_to_melt = df_user_items2.columns[:9]

# Utiliza melt directamente con las columnas a derretir
df_user_items2 = pd.melt(df_user_items2, id_vars=['user_id', 'user_url'], 
                         value_vars=columns_to_melt,
                         value_name='reviews')

df_user_items2.head()

,user_id,user_url,variable,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,0,"{'funny': '', 'posted': 'Posted November 5, 20..."
1,js41637,http://steamcommunity.com/id/js41637,0,"{'funny': '', 'posted': 'Posted June 24, 2014...."
2,evcentric,http://steamcommunity.com/id/evcentric,0,"{'funny': '', 'posted': 'Posted February 3.', ..."
3,doctr,http://steamcommunity.com/id/doctr,0,"{'funny': '', 'posted': 'Posted October 14, 20..."
4,maplemage,http://steamcommunity.com/id/maplemage,0,"{'funny': '3 people found this review funny', ..."


In [359]:
# Se utiliza pd.melt para transformar las columnas en filas conservando el 'user_id' y 'user_url'
df_user_items2 = pd.melt(df_user_items2, id_vars=['user_id', 'user_url'], 
                       value_vars=list(range(9)),
                       value_name='reviews')
df_user_items2.head()


ValueError: value_name (reviews) cannot match an element in the DataFrame columns.

In [ ]:
df_user_items2[df_user_items2['user_id']=='76561197970982479']

,user_id,user_url,variable,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,0,"{'funny': '', 'posted': 'Posted November 5, 20..."
25799,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1,"{'funny': '', 'posted': 'Posted July 15, 2011...."
51598,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2,"{'funny': '', 'posted': 'Posted April 21, 2011..."
77397,76561197970982479,http://steamcommunity.com/profiles/76561197970...,3,None
103196,76561197970982479,http://steamcommunity.com/profiles/76561197970...,4,None
128995,76561197970982479,http://steamcommunity.com/profiles/76561197970...,5,None
154794,76561197970982479,http://steamcommunity.com/profiles/76561197970...,6,None
180593,76561197970982479,http://steamcommunity.com/profiles/76561197970...,7,None
206392,76561197970982479,http://steamcommunity.com/profiles/76561197970...,8,None


In [ ]:
# Se eliminan las filas con valor None
df_user_items2 = df_user_items2.dropna()
# Se verifica que solo queden el 'user_id' con la cantidad de diccionarios que le corresponde
df_user_items2[df_user_items2['user_id']=='76561197970982479']

,user_id,user_url,variable,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,0,"{'funny': '', 'posted': 'Posted November 5, 20..."
25799,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1,"{'funny': '', 'posted': 'Posted July 15, 2011...."
51598,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2,"{'funny': '', 'posted': 'Posted April 21, 2011..."


In [ ]:
# Se separan por columnas cada una de las claves de 'reviews'
df_reviews = df_reviews2['reviews'].apply(pd.Series, dtype='object')
df_reviews = df_reviews.add_prefix('reviews_')
# Desplegar el DataFrame
df_reviews.head()

,reviews_funny,reviews_posted,reviews_last_edited,reviews_item_id,reviews_helpful,reviews_recommend,reviews_review
0,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
2,,Posted February 3.,,248820,No ratings yet,True,A suitably punishing roguelike platformer. Wi...
3,,"Posted October 14, 2013.",,250320,2 of 2 people (100%) found this review helpful,True,This game... is so fun. The fight sequences ha...
4,3 people found this review funny,"Posted April 15, 2014.",,211420,35 of 43 people (81%) found this review helpful,True,Git gud


In [ ]:
# Se une con el 'user_id' y 'user_url'
df_reviews = pd.concat([df_reviews2[['user_id', 'user_url']], df_reviews], axis=1)
df_reviews.head()

,user_id,user_url,reviews_funny,reviews_posted,reviews_last_edited,reviews_item_id,reviews_helpful,reviews_recommend,reviews_review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,js41637,http://steamcommunity.com/id/js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
2,evcentric,http://steamcommunity.com/id/evcentric,,Posted February 3.,,248820,No ratings yet,True,A suitably punishing roguelike platformer. Wi...
3,doctr,http://steamcommunity.com/id/doctr,,"Posted October 14, 2013.",,250320,2 of 2 people (100%) found this review helpful,True,This game... is so fun. The fight sequences ha...
4,maplemage,http://steamcommunity.com/id/maplemage,3 people found this review funny,"Posted April 15, 2014.",,211420,35 of 43 people (81%) found this review helpful,True,Git gud


In [ ]:
df_reviews['reviews_last_edited'][0]

''

In [ ]:
df_reviews.replace('', None, inplace=True)
df_reviews.head()

,user_id,user_url,reviews_funny,reviews_posted,reviews_last_edited,reviews_item_id,reviews_helpful,reviews_recommend,reviews_review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,None,"Posted November 5, 2011.",None,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,js41637,http://steamcommunity.com/id/js41637,None,"Posted June 24, 2014.",None,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
2,evcentric,http://steamcommunity.com/id/evcentric,None,Posted February 3.,None,248820,No ratings yet,True,A suitably punishing roguelike platformer. Wi...
3,doctr,http://steamcommunity.com/id/doctr,None,"Posted October 14, 2013.",None,250320,2 of 2 people (100%) found this review helpful,True,This game... is so fun. The fight sequences ha...
4,maplemage,http://steamcommunity.com/id/maplemage,3 people found this review funny,"Posted April 15, 2014.",None,211420,35 of 43 people (81%) found this review helpful,True,Git gud


In [ ]:
# Se eliminan las columnas 'reviews_funny' y 'reviews_last_edited'
df_reviews = df_reviews.drop(columns=['reviews_funny', 'reviews_last_edited'])
df_reviews.columns

Index(['user_id', 'user_url', 'reviews_posted', 'reviews_item_id',
       'reviews_helpful', 'reviews_recommend', 'reviews_review'],
      dtype='object')

# Transformacion de la columna 'review_posted'

In [ ]:

df_reviews['reviews_date']

KeyError: 'reviews_date'

In [ ]:
df_reviews[df_reviews['reviews_date'] == 'Formato inválido']

In [ ]:
df_reviews = df_reviews.drop('reviews_posted', axis=1)
df_reviews.columns

# Transformación de la columna 'reviews'

In [ ]:
df_reviews = df_reviews.dropna(subset=['reviews_review'])
# Se verifican nulos
utils.verificar_tipo_datos(df_reviews)

# Carga del dataset australian_user_reviews

In [ ]:
archivo_limpio = 'data/user_review_limpio.csv'
df_reviews.to_csv(archivo_limpio, index=False, encoding='utf-8')
print(f'Se guardó el archivo {archivo_limpio}')

# Transformar la columna 'reviews_posted'

# Columna 'reviews_review'

# Cargar el Dataset 'australian_user_reviews'

In [ ]:
archivo = 'Datasets/user_review_curado.csv'
df_reviews.to_csv(archivo_limpio, index=False, encoding='utf-8')
print(f'Se guardó el archivo {archivo_limpio}')